Basic theory of likelihood 
===================

This jupyter notebook gives a basic overview of the theory of likelihood and maximum likelihood estimators applied to *Fermi* LAT. 

In the introduction to some of the probabilistic concepts below, I was blatantly inspired by the excellent [Bayesian methods in astronomy](https://github.com/jakevdp/BayesianAstronomy) tutorial. In fact, I am copying parts of Jake Vanderplas' tutorial below. You should go and read it if you have the chance.

Let's begin with some Python imports:

In [1]:
%pylab inline
import seaborn # for plot formatting

Populating the interactive namespace from numpy and matplotlib


# Fundamental questions of statistics

There are two fundamental types of statistical questions we want to answer:

**1. Model Fitting:** *Given this Model, what parameters best fit my data?*

Examples:

- What are the slope and intercept of a line of best-fit?
- What is the frequency, amplitude, and phase of a sinusoidal fit?

**2. Model Selection:** *Given two potential Models, which better describes my data?*

Examples:

- Does a linear or quadratic fit describe our data better?

Often one of the two models is a *null hypothesis*, or a baseline model in which the effect you're interested in is not observed.

# Likelihood definition

Here, we will focus on frequentist *maximum likelihood* approaches as a way of performing both model fitting and model selection. Another approach is Bayesian methods, but given our time constraints we will not cover them.

The starting point of maximum likelihood methods is to define the probability of seeing our data given the model—the likelihood:

$$ P(data ~|~ scientific\ model) $$ 

Let's define some symbols that will let us express this more easily:

$$
P(D ~|~ \theta)
$$

- $\theta$ represents the "science": the set of parameters that we are interested in constraining
- $D$ represents the "observed data"

It makes sense that the best-fit parameters that describe the data are those that maximize the likelihood defined above. Now all we need to do--as far as likelihood methods are concerned--is to compute the likelihood and maximize it. This should give us a point estimate of the model parameters that best describe the data.

# Simple example of statistical model

Since we want to maximize the likelihood, we need an expression to compute $P(D ~|~ \theta)$ for our data as a function of the parameters $\theta$.



If we were given:

- Data points $x_i, y_i$ with simple errorbars—this implies that probability for any *single* data point is a normal distribution about the true value
- Model $y_M(x; \theta)$ providing expected values

then

$$
y_i \sim \mathcal{N}(y_M(x_i;\theta), \sigma)
$$

and the likelihood would be

$$
P(x_i,y_i\mid\theta) = \frac{1}{\sqrt{2\pi\varepsilon_i^2}} \exp\left(\frac{-\left[y_i - y_M(x_i;\theta)\right]^2}{2\varepsilon_i^2}\right)
$$

where $\varepsilon_i$ are the (known) measurement errors indicated by the errorbars.

Assuming all the points are independent, we can find the *full likelihood by multiplying the individual likelihoods together*:

$$
P(D\mid\theta) = \prod_{i=1}^N P(x_i,y_i\mid\theta)
$$

which is a function of the model parameters and the data. From now on, we will refer to the likelihood function as $\mathcal{L}$:

$$ \mathcal{L} \equiv  P(D\mid\theta). $$

For convenience (and also for numerical accuracy) this is often expressed in terms of the *log-likelihood*, which for our simple example is:

$$
\log \mathcal{L} = \log P(D\mid\theta) = -\frac{1}{2}\sum_{i=1}^N\left(\log(2\pi\varepsilon_i^2) + \frac{\left[y_i - y_M(x_i;\theta)\right]^2}{\varepsilon_i^2}\right).
$$

### Exercises

1. Write a python method that creates some Mock data with errors bars, given a known line (i.e. known slope and intercept). We will fit this data below in order to recover the line parameters.

2. Write a Python function which computes the log-likelihood given a parameter vector $\theta$, an array of errors $\varepsilon$, and an array of $x$ and $y$ values

3. Use tools in [`scipy.optimize`](http://docs.scipy.org/doc/scipy/reference/optimize.html) to maximize this likelihood (i.e. minimize the negative log-likelihood). How close is this result to the input ``theta_true`` that you provided in exercise 1?

## Useful references

- Bevington, Data reduction and analysis for the physical sciences
- Lyons, Statistics for nuclear and particle physicists

# Likelihood for *Fermi* LAT data

In our case, the input model is the distribution of gamma-ray sources on the sky and includes their intensity and spectra. We will bin the data in multidimensional bins—energy, sky pixels, time etc—because the counts are characterized by many variables. Thus, even with many counts, each bin will contain a small number of counts.

The observed number of counts in each bin $i$ is characterized by the [Poisson distribution](https://en.wikipedia.org/wiki/Poisson_distribution). With a small number of counts per bin, the Poisson distribution cannot be approximated by a normal distribution.

The likelihood $\mathcal{L}$ is the product of the probabilities of observing the detected counts in each bin. The detected counts in each bin is $n_i$, while the number of expected counts predicted by the model in the $i$th bin is $m_i$. Note that $m_i$ is a function of the source model, and will differ for different models. 

### Exercise 4

Assuming a Poisson distribution, write down the probability of detecting $n_i$ counts in a bin $i$, given the expected number of counts $m_i$. Try to work it out before proceeding with the text.

- - - 

The probability of detecting $n_i$ counts in this bin is $P_i={m_i}^{n_i} e^{-m_i}/n_i!$. The likelihood $\mathcal{L}$ is the product of the probabilities $P_i$ of observing the detected counts in each bin for all $i$:

$$
\mathcal{L} = \prod_i P_i = \prod_i \frac{m_i^{n_i} e^{-m_i}}{n_i !}
$$

Using the properties of the product, $\mathcal{L}$ can be written in a slightly more convenient way:

- - -
**Binned likelihood**
$$\mathcal{L} = \prod_i e^{-m_i} \prod_i \frac{m_i^{n_i}}{n_i !} = e^{-N_{\rm pred}} \prod_i \frac{m_i^{n_i}}{n_i !} $$
- - - 

where $N_{\rm pred}$ is total number of counts that the source model predicts should have been detected.

The expression we just derived above is factored into $e^{-N_{\rm pred}}$ which is purely a function of the source model, and the product of $m_i^{n_i}/n_i!$ which is a function of both the source model and the data.

This likelihood, with finite size bins and $n_i$ that may be greater than 1, is the basis for **binned likelihood**. Since binning destroys information (i.e., the precise values of the quantities describing a count), there is a tradeoff between the number of bins (and thus the bin size) and the accuracy; smaller bins result in a more accurate likelihood.

If we let the bin sizes get infinitesmally small, then $n_i=0$ or 1. The likelihood is now 

- - - 
**Unbinned likelihood**
$$\mathcal{L} = e^{-N_{\rm pred}} \prod_i m_i $$
- - - 

Since $m_i$ is calculated using the precise values for each count—and not an average over a finite size bin—this **unbinned likelihood** is the most accurate.

For a small number of counts the unbinned likelihood can be calculated rapidly, but as the number of counts increases the time to calculate the likelihood becomes prohibitive, and the binned likelihood must be used.

One will maximize $\mathcal{L}$ to get the best match of the model to the data.

MLE for Fermi 

How to do it in practice: tutorial 

A number of steps are necessary to fit a source's spectra; these are described in detail below.

1. Select the data. The data from a substantial spatial region around the source(s) being analyzed must be used because of the overlapping of the point spread functions of nearby sources.
2. Select the model. This model includes the position of the source(s) being analyzed, the position of nearby sources, a model of the diffuse emission, the functional form of the source spectra, and values of the spectral parameters. In fitting the source(s) of interest, you will let the parameters for these sources vary, but because the region around these sources includes counts from nearby sources in which you are not interested, you might also let the parameters from these nearby sources vary.
3. Precompute a number of quantities that are part of the likelihood computation. As the parameter values are varied in searching for the best fit, the likelihood is calculated many times. While not strictly necessary, precomputing a number of computation-intensive quantities will greatly speed up the fitting process.
4. Finally, perform the actual fit. The parameter space can be quite large—the spectral parameters from a number of sources must be fit simultaneously—and therefore the likelihood tools provide a choice of three 'optimizers' (section 7.8) to maximize the likelihood efficiently. Fitting requires repeatedly calculating the likelihood for different trial parameter sets until a value sufficiently near the maximum is found; the optimizers guide the choice of new trial parameter sets to converge efficiently on the best set. The variation of the likelihood in the vicinity of the maximum can be related to the uncertainties on the parameters, and therefore these optimizers estimate the parameter uncertainties.

## Useful references

- [Likelihood overview](https://fermi.gsfc.nasa.gov/ssc/data/analysis/documentation/Cicerone/Cicerone_Likelihood/Likelihood_overview.html), FSSC

[Solutions to exercises](fermi_likelihood_lecture-solutions.ipynb).